# Process US County-level Data from NYT

In [20]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read data and get basic county groups

0.0 Report Data

In [24]:
# Read updated CSV file
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.string_})
# Change 'Unknown' fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
# Change 'Unknown' deaths to 0 and death column to int32 type
df_counties_original['deaths'] = df_counties_original['deaths'].replace({np.nan: 0})
df_counties_original = df_counties_original.astype({'deaths': 'int32'})
df_counties_original.sort_values("fips", ascending=True)

,date,county,state,fips,cases,deaths
724935,2020-11-12,Unknown,Puerto Rico,00000,2420,909
103556,2020-04-30,Unknown,Vermont,00000,3,0
103563,2020-04-30,Unknown,Virgin Islands,00000,0,4
592749,2020-10-02,Unknown,Wisconsin,00000,0,10
735607,2020-11-16,Unknown,Alaska,00000,199,0
...,...,...,...,...,...,...
190396,2020-05-29,St. Thomas,Virgin Islands,78030,38,0
537389,2020-09-15,St. Thomas,Virgin Islands,78030,744,12
514737,2020-09-08,St. Thomas,Virgin Islands,78030,712,11
670312,2020-10-26,St. Thomas,Virgin Islands,78030,779,14


In [25]:
df_counties_original.dtypes

date      object
county    object
state     object
fips      object
cases      int32
deaths     int32
dtype: object

0.1 GeoJSON Data

In [ ]:
# Read GeoJson data
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)

0.2 Group Data to Counties

In [ ]:
# Transform cases/deaths data table to pivot table using fips as main index
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties

0.3 Get Date Info

In [ ]:
# Get dates from data table
date= pivot_counties['cases'].columns
date

In [ ]:
# Sort dates and get the start/end
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [ ]:
# Format dates and create a list
start = datetime.strptime(dt_str_start, "%Y-%m-%d")
end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=start,end=end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [ ]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

# Create time series of cases/deaths
pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()

In [ ]:
# Get the date of first case/death
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
# For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [ ]:
# Get the latest case/death
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

Step3:  
Extract necessary columns

In [ ]:
# Only keep the necessary columns
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

Step4:  
Assign geometries to counties

In [ ]:
# Merge case dataframe and geometry
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "GEOID", "population", "geometry"]], report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Step5:  
Deal with nan values

In [ ]:
# Replace NAN values with 0
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)
final_df['fips'] = final_df['fips'].replace(np.nan, final_df['GEOID'])

In [ ]:
# Set up template
template =final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]

In [ ]:
for x in range(0,len(template)):
    template[x] = 0
template = ','.join(map(str, template))

In [ ]:
# Change list format to string with commas
final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df.head(5)

Step6:  
Finalize the dataframe

In [ ]:
# Keep and add essential columns
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 
                   'today_case','today_new_case','today_death','today_new_death', "geometry"]]
final_df['start'] = dt_str_start
final_df['end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(1)

In [ ]:
# Calculate weekly change rates of cases and form a time-series list
final_df['change_ts'] = ""
for x in range(0, len(final_df)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = final_df.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
#     print(','.join(changeRateStrList))
    final_df.loc[x,'change_ts'] = ','.join(changeRateStrList)

final_df

Step 7:  
Output file

In [ ]:
# Save file
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")